In [16]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re

In [2]:
crash_data = '/Users/alex/Github/nyc-crashes/src/data/crashes-20201220-215930.csv'
hospitals_data = '/Users/alex/Github/nyc-crashes/src/data/hospitals-20201220-215939.csv'
departments_data = '/Users/alex/Github/nyc-crashes/src/data/police_departments_data.csv'

crash_data = pd.read_csv(crash_data, low_memory=False)
hospitals_data = pd.read_csv(hospitals_data, low_memory=False)
departments_data = pd.read_csv(departments_data, low_memory=False)

In [3]:
from datetime import datetime, timedelta

yesterday = datetime.today() - timedelta(days=6)
last_week = [(yesterday - timedelta(days=x)).strftime('%m/%d/%Y') for x in range(7)]
last_month = [(yesterday - timedelta(days=x)).strftime('%m/%d/%Y') for x in range(30)]
yesterday = yesterday.strftime('%m/%d/%Y')

In [4]:
yesterday_data = crash_data[crash_data["CRASH DATE"] == yesterday]
last_week_data = crash_data[crash_data["CRASH DATE"].isin(last_week)]
last_month_data = crash_data[crash_data["CRASH DATE"].isin(last_month)]

# Visualization

In [11]:
yesterday_w_loc = yesterday_data.dropna(subset=['LOCATION'])
yesterday_w_loc.head(1)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
45407,12/15/2020,13:50,BROOKLYN,11236,40.648266,-73.90581,"(40.648266, -73.90581)",ROCKAWAY PARKWAY,FOSTER AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4376388,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN


In [12]:
hospitals_data.dropna(subset=['Location 1'], inplace=True)
hospitals_data.drop_duplicates(subset=['Location 1'], inplace=True)
hospitals_data.head(1)

,Facility Type,Borough,Facility Name,Cross Streets,Phone,Location 1,Postcode,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,Child Health Center,Manhattan,La Clinica Del Barrio,NaN,212-410-7940,"413 120th StreetNew York, NY 10035(40.79820504...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
departments_data.head(1)

,precinct,phone,address,location
0,1st Precinct,212-334-0611,16 Ericsson Place,"(40.7205994, -74.0083416)"


In [18]:
crash_times = yesterday_w_loc['CRASH TIME'].tolist()
streets = yesterday_w_loc['ON STREET NAME'].tolist()
yesterday_coords = yesterday_w_loc['LOCATION'].apply(lambda x: 
                                        [float(coord) for coord in x[1:-1].split(', ')]).tolist()

loc_extract_reg = re.compile("\(\d{2}\.\d+\,\s\-\d{2}\.\d+\)")

hospitals_data['Location 1'] = hospitals_data['Location 1'].apply(lambda x: loc_extract_reg.search(x).group(0))

hospital_names = hospitals_data['Facility Name'].tolist()
hospital_coords = hospitals_data['Location 1'].apply(lambda x:
                                        [float(coord) for coord in x.split('\n')[-1][1:-1].split(', ')]).tolist()

In [19]:
import folium

ny_map = folium.Map(location=[40.7306, -73.9352])

#add accidents

for crash_time, street, coord in zip(crash_times, streets, yesterday_coords):
    popup_msg = f"Crash Time: {crash_time}, Street: {street}"
    popup = folium.Popup(popup_msg, max_width=300)
    
    folium.Marker(
        location=coord,
        popup=popup,
        icon=folium.Icon(color='red', icon='info', prefix='fa')
    ).add_to(ny_map)
    
# add hospitals    

for name, coord in zip(hospital_names, hospital_coords):
    popup_msg = f"Hospital: {name}"
    popup = folium.Popup(popup_msg, max_width=300)
    
    folium.Marker(
        location=coord,
        popup=popup,
        icon=folium.Icon(color='green', icon='ambulance', prefix='fa')
    ).add_to(ny_map)
    
# add departments

for row in departments_data.itertuples():
    precinct = row.precinct
    phone = row.phone
    location = row.location
    location = [float(coord) for coord in location[1:-1].split(', ')]

    popup_msg = f"Precinct: {precinct}, Phone number: {phone}"
    popup = folium.Popup(popup_msg, max_width=300)

    folium.Marker(
        location=location,
        popup=popup,
        icon=folium.Icon(color='blue', icon='user', prefix='fa')
    ).add_to(ny_map)
        
ny_map

In [215]:
ny_map.save('map.html')